# Image Colorization
This notebook contains code used to scale up our images to 224 by 224 pixels and to colorize them with Deep AI's image colorizer API.

In [1]:
from fastbook import *
from fastai.vision.all import *
import pandas as pd
from PIL import Image
import numpy
import requests
import shutil
import os
from joblib import Parallel, delayed
import multiprocessing

path = '/home/cp47/CS344FinalProject/data/'
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
df = pd.read_csv(path + 'icml_face_data.csv')

def image_transform(img, label, index, train):
    img.resize((224,224)).save('temp.jpg')
    r = requests.post(
        "https://api.deepai.org/api/colorizer",
        files={
            'image': open('temp.jpg', 'rb'),
        },
        headers={'api-key': '9047e654-38d7-4ce6-9c9f-c2958c4fbe3c'}
    )
    if train:
        save_path = Path("data/colored-big/train/" + label + "/" + str(index) + ".jpg")
    else:
        save_path = Path("data/colored-big/test/" + label + "/" + str(index) + ".jpg")
    colorized_image_url = requests.get(r.json()["output_url"], stream=True)
    if colorized_image_url.status_code == 200:
        with open(save_path, "wb") as save_file:
            colorized_image_url.raw.decode_content = True
            shutil.copyfileobj(colorized_image_url.raw, save_file)
    else:
        print(f"image result status code: {colorized_image_url.status_code}")
        
def transform(x):
    img = np.fromstring(df[' pixels'][x], dtype=int, sep=' ')
    img = np.reshape(img, (48, 48)).astype(np.uint8)
    img = Image.fromarray(img)
    image_transform(img, emotions[df['emotion'][x]], x, df[' Usage'][x] == 'Training')

In [5]:
for x in range (df.shape[0]): transform(x)  # non-parallel method

KeyboardInterrupt: 

In [5]:
num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(transform)(x) for x in range (df.shape[0])) # parallel method

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [6]:
if os.path.exists("temp.jpg"):  # clean up
  os.remove("temp.jpg")